Import libraries required

In [171]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import difflib

Data collection and preprocessing

In [172]:
#Inputs our dataset into a datframe
movie_set = pd.read_csv('moviestmdbdataset.csv')

In [173]:
#Outputs the first 5 rows of our data set
movie_set.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [174]:
#Number of rows and columns
movie_set.shape

(4803, 24)

Feature Selection

In [175]:
# selecting features with useful data to convert to vectors
features_used = ['genres','keywords','overview','tagline','cast','director']
movie_set[features_used]

,genres,keywords,overview,tagline,cast,director
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,Following the death of District Attorney Harve...,The Legend Ends,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"John Carter is a war-weary, former military ca...","Lost in our world, found in another.",Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton
...,...,...,...,...,...,...
4798,Action Crime Thriller,united states\u2013mexico barrier legs arms pa...,El Mariachi just wants to play his guitar and ...,"He didn't come looking for trouble, but troubl...",Carlos Gallardo Jaime de Hoyos Peter Marquardt...,Robert Rodriguez
4799,Comedy Romance,NaN,A newlywed couple's honeymoon is upended by th...,A newlywed couple's honeymoon is upended by th...,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,Edward Burns
4800,Comedy Drama Romance TV Movie,date love at first sight narration investigati...,"""Signed, Sealed, Delivered"" introduces a dedic...",NaN,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,Scott Smith
4801,NaN,NaN,When ambitious New York attorney Sam is sent t...,A New Yorker in Shanghai,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,Daniel Hsia


In [176]:
for attribute in features_used:
    movie_set[attribute] = movie_set[attribute].fillna('')

In [177]:
#combined attributes
comb_att = movie_set['genres']+' '+movie_set['keywords']+' '+movie_set['overview']+' '+ movie_set['tagline'] +' '+ movie_set['cast'] +' '+ movie_set['director']

In [178]:
print(comb_att)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      When ambitious New York attorney Sam is sent...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


Convert the data into vectors

In [179]:
vectorizer = TfidfVectorizer()

In [180]:
feature_vectors = vectorizer.fit_transform(comb_att)

In [181]:
print(feature_vectors)

  (0, 4288)	0.13213124585063996
  (0, 14180)	0.08418056281586364
  (0, 23134)	0.1498786462809525
  (0, 17784)	0.12285843797047787
  (0, 15569)	0.171366691592621
  (0, 25868)	0.11590870590502833
  (0, 29614)	0.15064979633862852
  (0, 24802)	0.158925784923944
  (0, 23578)	0.16802034155542864
  (0, 30543)	0.15064979633862852
  (0, 30145)	0.18243919685121024
  (0, 23619)	0.11047375838269538
  (0, 19410)	0.0318696249813729
  (0, 30121)	0.07272849549697494
  (0, 9065)	0.14505772980982928
  (0, 5245)	0.16506865163441015
  (0, 1014)	0.12285843797047787
  (0, 1243)	0.05270709034262741
  (0, 21523)	0.1665010643166998
  (0, 1281)	0.03195190270050669
  (0, 19650)	0.1612055740237117
  (0, 10368)	0.13848277654301597
  (0, 2974)	0.09974129089792806
  (0, 27515)	0.14569220419090678
  (0, 2685)	0.09805341831299116
  :	:
  (4802, 27153)	0.05940079357098276
  (4802, 19243)	0.061349419053434766
  (4802, 11941)	0.07829183421073846
  (4802, 29680)	0.07735547381237222
  (4802, 9346)	0.07104321122746965
  (48

Getting the similariity score of vectors

In [182]:
sim_list = cosine_similarity(feature_vectors)

In [183]:
print(sim_list)

[[1.         0.05083168 0.0332947  ... 0.02749812 0.0304889  0.0072518 ]
 [0.05083168 1.         0.04356836 ... 0.05077045 0.03100979 0.01521198]
 [0.0332947  0.04356836 1.         ... 0.02646984 0.04751623 0.01372603]
 ...
 [0.02749812 0.05077045 0.02646984 ... 1.         0.03481447 0.03546821]
 [0.0304889  0.03100979 0.04751623 ... 0.03481447 1.         0.03098945]
 [0.0072518  0.01521198 0.01372603 ... 0.03546821 0.03098945 1.        ]]


In [184]:
sim_list.shape

(4803, 4803)

In [185]:
movie_name = input("What's your favourite movie?: ")

What's your favourite movie?: the dark knight


In [186]:
all_titles_list = movie_set['title'].tolist()
print(all_titles_list)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

Finding an approximate match to input

In [187]:
find_close_match = difflib.get_close_matches(movie_name, all_titles_list)
print(find_close_match)

['The Dark Knight', 'Shark Night', 'The Dark Knight Rises']


In [188]:
best_match = find_close_match[0]

In [189]:
print(best_match)

The Dark Knight


Finding index of match

In [190]:
movie_index = movie_set[movie_set.title == best_match]['index'].values[0]
print(movie_index)

65


Finding similar movies 

In [191]:
similarity_score = list(enumerate(sim_list[movie_index]))
print(similarity_score)

[(0, 0.03276531641236893), (1, 0.05082953606908145), (2, 0.051047931711705886), (3, 0.3672655963826401), (4, 0.03629076283233519), (5, 0.032575984733848935), (6, 0.042607499349181184), (7, 0.05439369497648243), (8, 0.024618119449957353), (9, 0.12091463837042632), (10, 0.07378027069129206), (11, 0.05645153163196112), (12, 0.020228503924470086), (13, 0.03691433130266145), (14, 0.08316718269626229), (15, 0.0489045582416184), (16, 0.042418919175667216), (17, 0.03644025669089048), (18, 0.06834000664008105), (19, 0.050091195401632205), (20, 0.05765503738155681), (21, 0.040250437662658685), (22, 0.026249837356468374), (23, 0.027171527896929788), (24, 0.030827889869180898), (25, 0.03332116945479635), (26, 0.03904463316893502), (27, 0.050268382047400695), (28, 0.018385948571022465), (29, 0.05985954428503216), (30, 0.05452492847234437), (31, 0.02971055453495607), (32, 0.041422942386203636), (33, 0.02146564417494728), (34, 0.016172850511589947), (35, 0.050030642617715365), (36, 0.0568276994433194

Sort based on similarity_score

In [192]:
sorted_similar = sorted(similarity_score, key = lambda x:x[1], reverse = True)

In [193]:
print(sorted_similar)

[(65, 1.0000000000000002), (3, 0.3672655963826401), (119, 0.3088318148668579), (428, 0.2590603581207476), (3854, 0.2377185637540214), (299, 0.22593653462189475), (1359, 0.19739154749412366), (210, 0.15316341851977347), (9, 0.12091463837042632), (3725, 0.1203965533711609), (1196, 0.11636548954898629), (205, 0.11544115886560387), (1433, 0.11436853417139027), (2020, 0.10814005612879954), (3819, 0.1034030069448775), (4099, 0.10218349813025428), (140, 0.10201696047898216), (879, 0.100867046110802), (3454, 0.10051491635220137), (4135, 0.09972491711730949), (58, 0.09820560391555169), (2308, 0.09787597705316793), (2147, 0.09702296869401279), (246, 0.0962277060777231), (582, 0.09571391456684646), (163, 0.09557508863088111), (2160, 0.09428343756131405), (732, 0.09367502356704006), (1740, 0.09297121289005787), (72, 0.09289880722384858), (421, 0.09235198580069304), (4759, 0.09187547459981969), (2282, 0.09181726923500778), (1720, 0.09161983609367585), (870, 0.09157248430242707), (1084, 0.0915344894

Print names of movies that are recommended

In [197]:
print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar:
  index = movie[0]
  title_of_index = movie_set[movie_set.index==index]['title'].values[0]
  if (i<21):
    print(i, '.',title_of_index)
    i+=1

Movies suggested for you : 

1 . The Dark Knight
2 . The Dark Knight Rises
3 . Batman Begins
4 . Batman Returns
5 . Batman: The Dark Knight Returns, Part 2
6 . Batman Forever
7 . Batman
8 . Batman & Robin
9 . Batman v Superman: Dawn of Justice
10 . The Sweeney
11 . The Prestige
12 . Sherlock Holmes: A Game of Shadows
13 . The Four Feathers
14 . The Rookie
15 . Defendor
16 . Harsh Times
17 . White House Down
18 . Law Abiding Citizen
19 . The Usual Suspects
20 . Gangster's Paradise: Jerusalema
